In [5]:
import json
import oandapyV20
from oandapyV20 import API   
from oandapyV20.endpoints import instruments, accounts, orders, trades, positions
import oandapyV20.endpoints.pricing as pricing
import pandas as pd
import numpy as np
import mplcyberpunk
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import schedule
import time
plt.style.use("cyberpunk")

In [6]:
ACCESS_TOKEN = "4aa51e7711418ba1aa356b835fab3afd-53be4f4d5aa21768bf421b354527dc96"
ACCOUNT_ID = "101-004-16909090-001"
client = API(access_token=ACCESS_TOKEN)

In [40]:
def get_prices(to_date: str, count: int = 3,instrument: str = "EUR_USD", granularity: str = "H1"):  
  params = {
    "count": count,
    "granularity": granularity,
    "to": to_date
    }
  r = instruments.InstrumentsCandles(instrument=instrument, params=params)
  return client.request(r)


def make_df(data: dict):
  data = pd.DataFrame(data["candles"])
  candles = pd.json_normalize(data["mid"]).rename(columns={"o": "open", "h": "high", "l": "low", "c": "close"})
  data = data.join(candles, how = "left").drop(["mid"], axis = 1)
  data["time"] = pd.to_datetime(data["time"])
  data = data.apply(lambda x: x.astype(float) if x.name in ["open", "high", "low", "close"] else x, axis = 0)
  return data

def get_to_date_in_utc(day_diff: int = 0):
    return (datetime.utcnow().replace(microsecond=0,second=0,minute=0) - timedelta(days=day_diff)).strftime("%Y-%m-%dT%H:%M:%S.%fZ")

def is_market_open():
  return datetime.now().weekday() not in [5, 6]

def get_signal(data):
  data["1h_spread"] = data["close"].diff()
  data["ma2"] = data["1h_spread"].rolling(window=2).mean()
  data["signal"] = data[["1h_spread","ma2"]].mean(axis=1)
  return np.sign(data["signal"].iloc[-1])


In [8]:
to_date = get_to_date_in_utc(day_diff=2)
print(to_date)
data = make_df(
    get_prices(to_date=to_date)
    )

2024-05-24T21:00:00.000000Z


In [39]:
def trade():
    if is_market_open():
        open_trade_request = trades.OpenTrades(accountID=ACCOUNT_ID)
        open_trade = client.request(open_trade_request)
        open_trade = open_trade["trades"]
        open_trade_request = trades.OpenTrades(accountID=ACCOUNT_ID)
        open_trade = client.request(open_trade_request)
        open_trade = open_trade["trades"]
        if open_trade:
            # Get the ID of the first open trade
            trade_id = open_trade[0]["id"]
            current_units = open_trade[0]["currentUnits"]
            
            # Step 2: Close the trade
            close_trade_request = trades.TradeClose(accountID=ACCOUNT_ID, tradeID=trade_id)
            close_trade_response = client.request(close_trade_request)
            
            # Print the response
            print(f"Trade closed: {close_trade_response}")
        else:
            to_date = get_to_date_in_utc()
            data = make_df(
                get_prices(to_date=to_date)
                )
            signal = get_signal(data)
            quantity = 10000 if signal > 0 else -10000
            # Order details
            order_data = {
                "order": {
                    "instrument": "EUR_USD",  # Replace with your desired instrument
                    "units": quantity,  # Number of units to buy (positive for buy, negative for sell)
                    "type": "MARKET",  # Order type
                    "positionFill": "DEFAULT"  # Order position fill option
                }
            }
            trade_request = orders.OrderCreate(accountID=ACCOUNT_ID, data=order_data)
            trade = client.request(trade_request)
            print(f"Trade opened: {trade}")

In [38]:
schedule.every(4).hours.at(":00").do(trade)
while True:
    # Run all pending jobs
    schedule.run_pending()
    time.sleep(1)

Trade closed: {'orderCreateTransaction': {'id': '1046', 'accountID': '101-004-16909090-001', 'userID': 16909090, 'batchID': '1046', 'requestID': '97272620288504791', 'time': '2024-05-26T22:43:54.552346218Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '10000', 'timeInForce': 'FOK', 'positionFill': 'REDUCE_ONLY', 'reason': 'TRADE_CLOSE', 'tradeClose': {'units': 'ALL', 'tradeID': '1044'}}, 'orderFillTransaction': {'id': '1047', 'accountID': '101-004-16909090-001', 'userID': 16909090, 'batchID': '1046', 'requestID': '97272620288504791', 'time': '2024-05-26T22:43:54.552346218Z', 'type': 'ORDER_FILL', 'orderID': '1046', 'instrument': 'EUR_USD', 'units': '10000', 'requestedUnits': '10000', 'price': '1.08509', 'pl': '-0.7410', 'quotePL': '-0.80000', 'financing': '0.0000', 'baseFinancing': '0.00000000000000', 'commission': '0.0000', 'accountBalance': '100067.4902', 'gainQuoteHomeConversionFactor': '0.917004211541', 'lossQuoteHomeConversionFactor': '0.92622033427', 'guaranteedExec

In [35]:
int(open_trade[0]["currentUnits"])

-10000